In [ ]:
pip install streamlit

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import svm
import streamlit as st
import pickle

In [ ]:
# Muat dataset
df = pd.read_csv('/content/drive/MyDrive/DATASET DATA MINING/Sleep_health_and_lifestyle_dataset.csv')

In [ ]:
df.head()


,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [ ]:
df.isnull().sum()

Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64

In [ ]:
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           374 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [ ]:
df['Gender'].value_counts()

Gender
Male      189
Female    185
Name: count, dtype: int64

In [ ]:
df['Occupation'].value_counts()

Occupation
Nurse                   73
Doctor                  71
Engineer                63
Lawyer                  47
Teacher                 40
Accountant              37
Salesperson             32
Software Engineer        4
Scientist                4
Sales Representative     2
Manager                  1
Name: count, dtype: int64

In [ ]:
df['BMI Category'].value_counts()

BMI Category
Normal           195
Overweight       148
Normal Weight     21
Obese             10
Name: count, dtype: int64

In [ ]:
df['Blood Pressure'].value_counts()

Blood Pressure
130/85    99
140/95    65
125/80    65
120/80    45
115/75    32
135/90    27
140/90     4
125/82     4
132/87     3
128/85     3
126/83     2
115/78     2
139/91     2
142/92     2
119/77     2
135/88     2
129/84     2
128/84     2
131/86     2
117/76     2
130/86     2
118/75     2
121/79     1
122/80     1
118/76     1
Name: count, dtype: int64

In [ ]:
df['Sleep Disorder'].value_counts()

Sleep Disorder
None           219
Sleep Apnea     78
Insomnia        77
Name: count, dtype: int64

In [ ]:
# Membuat mapping untuk pelabelan manual
gender_mapping = {'Male': 0, 'Female': 1}

occupation_mapping = {
    'Nurse': 0, 'Doctor': 1, 'Engineer': 2, 'Lawyer': 3, 'Teacher': 4,
    'Accountant': 5, 'Salesperson': 6, 'Software Engineer': 7, 'Scientist': 8,
    'Sales Representative': 9, 'Manager': 10
}

bmi_category_mapping = {'Normal': 0, 'Overweight': 1, 'Normal Weight': 2, 'Obese': 3}

blood_pressure_mapping = {
    "115/75": 1, "115/78": 2, "117/76": 3, "118/75": 4, "118/76": 5,
    "119/77": 6, "120/80": 7, "121/79": 8, "122/80": 9, "125/80": 10,
    "125/82": 11, "126/83": 12, "128/84": 13, "128/85": 14, "129/84": 15,
    "130/85": 16, "130/86": 17, "131/86": 18, "132/87": 19, "135/88": 20,
    "135/90": 21, "139/91": 22, "140/90": 23, "140/95": 24, "142/92": 25
}

sleep_disorder_mapping = {'None': 0, 'Sleep Apnea': 1, 'Insomnia':2}

# Melakukan pelabelan manual menggunakan metode .map()
df['Gender'] = df['Gender'].map(gender_mapping)
df['Occupation'] = df['Occupation'].map(occupation_mapping)
df['BMI Category'] = df['BMI Category'].map(bmi_category_mapping)
df['Blood Pressure'] = df['Blood Pressure'].map(blood_pressure_mapping)
df['Sleep Disorder'] = df['Sleep Disorder'].map(sleep_disorder_mapping)

df


,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,0,27,7,6.1,6,42,6,1,12,77,4200,0
1,2,0,28,1,6.2,6,60,8,0,10,75,10000,0
2,3,0,28,1,6.2,6,60,8,0,10,75,10000,0
3,4,0,28,9,5.9,4,30,8,3,23,85,3000,1
4,5,0,28,9,5.9,4,30,8,3,23,85,3000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,1,59,0,8.1,9,75,3,1,24,68,7000,1
370,371,1,59,0,8.0,9,75,3,1,24,68,7000,1
371,372,1,59,0,8.1,9,75,3,1,24,68,7000,1
372,373,1,59,0,8.1,9,75,3,1,24,68,7000,1


In [ ]:
df['Sleep Disorder'].value_counts()

Sleep Disorder
0    219
1     78
2     77
Name: count, dtype: int64

In [ ]:
X = df.drop(['Person ID', 'Sleep Disorder'], axis=1)
y = df['Sleep Disorder']

In [ ]:
# # Create a StandardScaler model
# scaler = StandardScaler()

# # Fit and transform the data
# df_scaled = scaler.fit_transform(X)

# # Convert the scaled data back to a DataFrame
# df_scaled = pd.DataFrame(df_scaled, columns=X.columns)

# # Print the first five rows of the scaled data
# print(df_scaled.head())

In [ ]:
# prompt: Masukkan lagi Sleep disorder ke df

#df_scaled['Sleep Disorder'] = df['Sleep Disorder']
#df_scaled.head()
# X = df.drop(['Person ID', 'Sleep Disorder'], axis=1)
# y = df['Sleep Disorder']
print(X)
print(y)


     Gender  Age  Occupation  Sleep Duration  Quality of Sleep  \
0         0   27           7             6.1                 6   
1         0   28           1             6.2                 6   
2         0   28           1             6.2                 6   
3         0   28           9             5.9                 4   
4         0   28           9             5.9                 4   
..      ...  ...         ...             ...               ...   
369       1   59           0             8.1                 9   
370       1   59           0             8.0                 9   
371       1   59           0             8.1                 9   
372       1   59           0             8.1                 9   
373       1   59           0             8.1                 9   

     Physical Activity Level  Stress Level  BMI Category  Blood Pressure  \
0                         42             6             1              12   
1                         60             8             

In [ ]:
from sklearn.svm import SVC

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(299, 11)
(75, 11)
(299,)
(75,)


In [ ]:
# Membuat model SVM
model = svm.SVC(kernel='linear')

# Melatih model
model.fit(X_train, y_train)

# Membuat prediksi
y_pred = model.predict(X_test)

# Menampilkan hasil prediksi
print(y_pred)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Menampilkan laporan klasifikasi
print(classification_report(y_test, y_pred))


[2 0 0 0 0 0 2 1 0 0 0 0 0 0 1 2 0 0 0 2 0 0 0 0 2 0 1 2 1 0 0 0 2 1 0 0 2
 2 1 0 0 0 0 0 2 0 0 0 2 1 2 0 0 0 0 1 1 1 0 0 0 0 0 1 0 2 0 0 0 0 2 0 0 1
 1]
Accuracy: 0.9066666666666666
              precision    recall  f1-score   support

           0       0.90      0.98      0.93        44
           1       1.00      0.81      0.90        16
           2       0.86      0.80      0.83        15

    accuracy                           0.91        75
   macro avg       0.92      0.86      0.89        75
weighted avg       0.91      0.91      0.91        75



In [ ]:
# Make a prediction with new data
input_data = [[1, 27, 9, 6.1, 6, 42, 6, 3, 11, 77, 4200]]  # Data yang hasil prediksi = 0
# input_data_as_numpy_array = np.asarray(input_data)
# input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)
# std_data = scaler.transform(input_data_reshaped)
# print(std_data)
prediction = model.predict(input_data)

print("Prediction:", prediction)
if prediction[0] == 0:
    print("Anda Tidak Mengalami Masalah Tidur")
elif prediction[0] == 1:
    print("Anda Mengalami SLEEP APNEA")
else:
    print("Anda Mengalami INSOMNIA")

Prediction: [0]
Anda Tidak Mengalami Masalah Tidur


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [ ]:
# simpan model
filename = 'sleep_quality_model.sav'
pickle.dump(model, open(filename, 'wb'))